<a href="https://colab.research.google.com/github/kevinvbc/gorilla/blob/main/Gorilla_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gorilla Data Engineer Assignment